In [1]:
import bliss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import pickle

<frozen importlib._bootstrap>:241: RuntimeWarning: nanobind: type 'ndarray' was already registered!

<frozen importlib._bootstrap>:241: RuntimeWarning: nanobind: type 'dev' was already registered!



In [2]:
# directory with M31 cadence from Choza et al. 2023

data_loc = '/datax/scratch/benjb/m31_l_data'

# pull cadence files into two separate lists
# due to bliss's unusual cadence input structure
# [[on, on, on], [off], [off], [off]]

on_files = glob.glob(f'{data_loc}/*MESSIER031*.h5')
off_files = list(set(glob.glob(f'{data_loc}/*.h5')) - set(on_files))

on_files.sort() # alphabetical order suffices for observation order
off_files.sort()

print('ON files:')
for file in on_files:
    print(file)

print('OFF files:')
for file in off_files:
    print(file)

# construct bliss input

cadence_files = []

cadence_files.append(on_files)

for file in off_files:
    cadence_files.append([file])

print('Elements of cadence input:')
for file in cadence_files:
    print(file)


ON files:
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5
OFF files:
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5
/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5
Elements of cadence input:
['/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5', '/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5', '/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5']
['/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5']
['/datax/scratch/benjb/m31_l_data/blc71_guppi_58832_17168_HIP3077_0060

In [3]:
# initialize cadence object

cadence = bliss.cadence(cadence_files)

In [4]:
# Run flaggers to remove coarse channel rolloff, bright RFI

# Filter rolloff
# Pass the cadence object and a float, the rolloff_width, in this example 20% of channels
cadence = bliss.flaggers.flag_filter_rolloff(cadence, .2)

# Spectral kurtosis calculated by frequency channel
# These are arbitrary for now, but the idea is to eliminate non-drifting signals and impulsive
# Interference, leaving in noise
cadence = bliss.flaggers.flag_spectral_kurtosis(cadence, .02, 20)

In [5]:
# Create noise estimation options structure to hold estimator method and mask choice
# Use mask, defaults to True
# Use standard deviation to estimate noise (other option Median Absolute Deviation)
noise_est_options = bliss.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True
noise_est_options.estimator_method = bliss.estimators.noise_power_estimator.stddev

# Run noise power estimators on cadence using flags and re-store in cadence object
cadence = bliss.estimators.estimate_noise_power(cadence, noise_est_options)

In [7]:
# Set drift rate parameters

# NOTE: Currently uses Fourier bins, listed as "rate" because
# it sets the implicit max absolute drift

# Again create an options structure, set the parameters as attributes of the structure
# Sets desmear to True! Will notice if a certain drift rate crosses bins between time steps, and set a desmearing bandwidth
# Defaults to cpu version
# Will have a future option to collect RFI for future filtering, counting which kinds of RFI appear along the trajectory where and of what kind (i.e. present in which flagging mask)
opts = bliss.integrate_drifts_options()
opts.desmear = True
opts.low_rate = -400
opts.high_rate = 400
opts.rate_step_size = 1


# Do the dedoppler integration, filling drift-frequency plane
cadence = bliss.drift_search.integrate_drifts(cadence, opts)

INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.0715Hz/sec
INFO: Searching drift rates from 4.0817Hz/sec to -4.071

In [8]:
# Create hit search options structure
# Allows setting SNR threshold, defaults to 10.0
# Two methods to set initial hit locations above a threshold: connected components and local maxima. Defaults to connected components
# Local maximum version allows setting a neighborhood
hit_search_opts = bliss.drift_search.hit_search_options()
hit_search_opts.method = bliss.drift_search.hit_search_methods.connected_components
hit_search_opts.snr_threshold = 10.0

# Perform the hit search! Returns a copy of cadence with hit field set
cadence = bliss.drift_search.hit_search(cadence, hit_search_opts)

In [ ]:
# Event search

# Event connection is done using a weighted distance metric accounting for SNR, signal power, difference in linear drift rates,
# and difference in frequency at extrapolated intersection point
# This is something we may want to test and chew on with setigen cadences
events = bliss.drift_search.event_search(cadence)

INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with M=16 and N=51
INFO: spec kurtosis with 

nt was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Event was found in an off scan
INFO: Even

In [27]:
# Writing outputs to files and reading them back in

# Can access individual lists of hits for each scan as class properties
# This, for example, will give the number of hits in the first ON target scan 
first_scan_hits = len(cadence.observations[0].scans[0].hits())
print(f"Number of hits in the first scan: {first_scan_hits}")


# Events is a list of event objects
# Interacting with event list directly in notebook

print(f"Number of events for this search: {len(events)}")
for e in events:
    print(f"{e.starting_frequency_Hz} : {e.average_drift_rate_Hz_per_sec} : {e.average_snr}")
    print(len(e.hits))
    print(e.hits)


# Write hits to file
# Produces a file for each scan in the cadence, labeled according to pointing location and by scan number
bliss.io.write_cadence_hits_to_files(cadence, "hits")

# Write events to file called "events_output"
bliss.io.write_events_to_file(events, "events_output")

# The files can then be read into scan, cadence, observation_target objects using respective methods
#read_hits = bliss.io.read_scan_hits_from_file("/home/benjb/bliss-notebooks/hits_obs0-unknown_0.cp")
# and events can be read similarly
read_events = bliss.io.read_events_from_file("/home/benjb/bliss-notebooks/events_output")
# print(read_events)

Number of hits in the first scan: 162
Number of events for this search: 4
1975001730.490476 : 0.0204085125464669 : 103.63499450683594
2
[hit: .start_freq_MHz=1975.001730 (.index=383877), .drift_rate_Hz_per_second=0.020409 (.index=46), .SNR=190.907303, .power=2776222208, bandwidth=114.6, hit: .start_freq_MHz=1975.000174 (.index=384415), .drift_rate_Hz_per_second=0.020409 (.index=46), .SNR=16.362688, .power=237610432, bandwidth=5.6]
1916669618.8040078 : 0.01020425627323345 : 132.8143310546875
2
[hit: .start_freq_MHz=1916.669619 (.index=290214), .drift_rate_Hz_per_second=0.010204 (.index=47), .SNR=252.999557, .power=3599712512, bandwidth=14.0, hit: .start_freq_MHz=1916.670323 (.index=289962), .drift_rate_Hz_per_second=0.010204 (.index=47), .SNR=12.629109, .power=179997120, bandwidth=8.4]
2062542771.4549005 : -0.01020425627323345 : 12.513575553894043
2
[hit: .start_freq_MHz=2062.542771 (.index=508979), .drift_rate_Hz_per_second=-0.010204 (.index=49), .SNR=13.241592, .power=288205440, bandw

In [2]:
read_hits = bliss.io.read_cadence_hits_from_files("/home/benjb/bliss-notebooks/hits")

In [32]:
bliss_hits_list = []

for observation in read_hits.observations:
    for scan in observation.scans:
        for hit in scan.hits():
            bliss_hits_list.append(str(hit))

print(len(bliss_hits_list))
for i in range(10):
    print(bliss_hits_list[i])

973
hit: .start_freq_MHz=2062.500175 (.index=524225), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=1479.371216, .power=31776686080, bandwidth=25.1
hit: .start_freq_MHz=2062.499999 (.index=524288), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=5538815.500000, .power=118972976463872, bandwidth=0.0
hit: .start_freq_MHz=2062.499825 (.index=524351), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=1473.290771, .power=31646078976, bandwidth=30.7
hit: .start_freq_MHz=2062.451342 (.index=541710), .drift_rate_Hz_per_second=0.397966 (.index=9), .SNR=18.131517, .power=224856256, bandwidth=86.6
hit: .start_freq_MHz=2062.499473 (.index=524476), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=209.816315, .power=4506825216, bandwidth=2.8
hit: .start_freq_MHz=2062.500524 (.index=524100), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=215.554901, .power=4630089216, bandwidth=5.6
hit: .start_freq_MHz=2062.462221 (.index=537807), .drift_rate_Hz_per_second=-0.265311 (.in

In [34]:
with open("bliss_hits", "wb") as fp:   #Pickling
   pickle.dump(bliss_hits_list, fp)

In [23]:
read_hits.observations[0].scans[0].hits()

[hit: .start_freq_MHz=2062.500175 (.index=524225), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=1479.371216, .power=31776686080, bandwidth=25.1,
 hit: .start_freq_MHz=2062.499999 (.index=524288), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=5538815.500000, .power=118972976463872, bandwidth=0.0,
 hit: .start_freq_MHz=2062.499825 (.index=524351), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=1473.290771, .power=31646078976, bandwidth=30.7,
 hit: .start_freq_MHz=2062.451342 (.index=541710), .drift_rate_Hz_per_second=0.397966 (.index=9), .SNR=18.131517, .power=224856256, bandwidth=86.6,
 hit: .start_freq_MHz=2062.499473 (.index=524476), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=209.816315, .power=4506825216, bandwidth=2.8,
 hit: .start_freq_MHz=2062.500524 (.index=524100), .drift_rate_Hz_per_second=-0.000000 (.index=48), .SNR=215.554901, .power=4630089216, bandwidth=5.6,
 hit: .start_freq_MHz=2062.462221 (.index=537807), .drift_rate_Hz_per_second=-0.26

In [21]:
help(read_hits.observations[0].scans[0].hits())

Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

In [10]:
read_hits = open('/home/benjb/bliss-notebooks/hits')
print(read_hits)
bliss.bliss_hits_to_dat(read_hits)

<_io.TextIOWrapper name='/home/benjb/bliss-notebooks/hits' mode='r' encoding='UTF-8'>


AttributeError: module 'bliss' has no attribute 'bliss_hits_to_dat'

In [5]:
with open("/home/benjb/bliss-notebooks/hits","r", 'capnp') as f:
   read_hits = f.read()
   print(read_hits)

TypeError: 'str' object cannot be interpreted as an integer

In [11]:
help(bliss.io.read_hits_from_file)

Help on nb_func in module bliss.pybliss.io:

read_hits_from_file = <nanobind.nb_func object>
    read_hits_from_file(arg: str, /) -> list[bliss.pybliss.drift_search.hit]



In [30]:
for event in read_events:
    print(event)

event: .starting_frequency_Hz=0 .average_drift_rate_Hz_per_sec=0.00.average_power=0 .average_snr=0 .event_start_seconds=0 .event_end_seconds=0
.hits=[{"hit: .start_freq_MHz=1975.001730 (.index=383877), .drift_rate_Hz_per_second=0.020409 (.index=46), .SNR=190.907303, .power=2776222208, bandwidth=114.6", "hit: .start_freq_MHz=1975.000174 (.index=384415), .drift_rate_Hz_per_second=0.020409 (.index=46), .SNR=16.362688, .power=237610432, bandwidth=5.6"}]
event: .starting_frequency_Hz=0 .average_drift_rate_Hz_per_sec=0.00.average_power=0 .average_snr=0 .event_start_seconds=0 .event_end_seconds=0
.hits=[{"hit: .start_freq_MHz=1916.669619 (.index=290214), .drift_rate_Hz_per_second=0.010204 (.index=47), .SNR=252.999557, .power=3599712512, bandwidth=14.0", "hit: .start_freq_MHz=1916.670323 (.index=289962), .drift_rate_Hz_per_second=0.010204 (.index=47), .SNR=12.629109, .power=179997120, bandwidth=8.4"}]
event: .starting_frequency_Hz=0 .average_drift_rate_Hz_per_sec=0.00.average_power=0 .average_